In [3]:


# =============================================================================
# CONSUMPTION 19 MALAWI VILLAGE
# =============================================================================
import numpy as np
import pandas as pd
import os 
os.chdir('C:/Users/rodri/Dropbox/JMP/python')
from data_functions_albert import remove_outliers, gini
os.chdir('C:/Users/rodri/Dropbox/Malawi/SIEG2021 (1)/2019 July/Data/Consumption')
percentiles = [0.05, 0.1, .25, .5, .75, 0.8, 0.9, 0.95, 0.99]
dollar_MWK = 734.24

import warnings
warnings.filterwarnings("ignore")

# =============================================================================
# Import data: Data from the field and conversion rates (ISA-LSMS price conversions)
# =============================================================================

data = pd.read_stata("raw/[2] Consumption+Non-Food Consumption_(9-16)_050819.dta", convert_categoricals=False)

##### Create conversion kg matrix(unitxitems) with the exact same names and units labels as in the data:


#item labels data
list_items = ['maizemgaiwa', 'maizerefined', 'maizemadeya', 'maizegrain', 'greenmaize', 'rice', 'cassavatubers', 'wsweetpotatoes', 'osweetpotatoes'
, 'ipotatoes', 'potatocrisps', 'bbean', 'pigeonpea', 'groundnut', 'groundnutf', 'onion', 'cabbage', 'tanaposi', 'leafyvegetables', 'tomato', 'eggs', 
'driedfish', 'fleshfish', 'goat', 'chicken', 'otherpoultry', 'smockedfish', 'mango', 'banana', 'guava', 
'wildfruits',  'sugar', 'sugarcane', 'cookingoil', 'softdrinks',
'thobwa', 'locallybrewed', 'salt', 'fingermillet', 'mandazidou']


# NOW I ALREADY CREATED IT THE CONVERSION FILE SO NO NEED TO RUN IT AGAIN ===============
'''
conversion_prod = pd.read_csv('conversions/Conversion_kg_prod_crops_IHS4.csv')

median_kg = conversion_prod.groupby(by=['unit', 'crop_code_data'])['conversion_rate_kg'].median()

## from the production side I take the crop-unit kg for the units we dont observe in consumption (50kg, Bale, Ox-cart)

data_conversion = pd.read_csv('Conversion_kg_IHS4.csv')
data_conversion = data_conversion[(data_conversion['hh_g04b_label'] != 'OTHER')]
data_conversion = data_conversion[(data_conversion['hh_g04b_label'] != 'OTHER (SPECIFY)')]


data_conversion.loc[data_conversion['hh_g04b_label']=='BASIN (SMALL)', 'unit'] = 21
data_conversion.loc[data_conversion['hh_g04b_label']=='SATCHET/TUBE (50G)', 'unit'] = 22
data_conversion.loc[data_conversion['hh_g04b_label']=='TIN 250G', 'unit'] = 17

data_conversion.columns = ['item_label', 'unit_label', 'unit', 'conversion_kgs']

conversion_units = data_conversion.groupby(by='unit')

items_conv = data_conversion.item_label.unique()

list_items_conv = ['Maize ufa mgaiwa (normal flour)',
       'Maize ufa refined (fine flour)', 'Maize ufa madeya (bran flour)',
       'Maize grain (not as ufa)', 'Green maize', 'Rice', 'Cassava tubers',
       'White sweet potato', 'Orange sweet potato',
       'Irish potato','Potato crisps',
       'Bean, brown', 'Pigeonpea (nandolo)', 'Groundnut fresh (unshelled)', 'Groundnut flour',
       'Onion', 'Cabbage', 'Tanaposi/Rape',
       'Other cultivated green leafy vegetables', 'Tomato', 'Eggs',
        'Sun Dried fish (Medium Variety)', 'Fresh fish (Medium Variety)', 'Goat', 'Chicken',
       'Other poultry - guinea fowl, doves, etc.', 'Smoked fish (Medium Variety)',  
       'Mango', 'Banana',  'Guava', 'Wild fruit (masau, malambe, etc.)', 'Sugar',
       'Sugar Cane', 'Cooking oil', 'Soft drinks (Coca-cola, Fanta, Sprite, etc.)', 'Thobwa',
       'Locally brewed liquor (kachasu)', 'Salt', 'Finger millet (mawere)',  'Mandazi, doughnut (vendor)' ]


conversionkg = data_conversion[data_conversion['item_label'].isin(list_items_conv)]

# Reshape as: rows:units, columns:crops
conversionkg = conversionkg.replace(list_items_conv, list_items)
conversionkg_pivot = conversionkg.pivot_table(values='conversion_kgs',
                                index='unit',
                                columns='item_label')
conversionkg_pivot.loc[99,:] = np.nan
conversionkg_pivot.loc[100,:] = np.nan

conversionkg_pivot.to_csv('conversionkg_isaprices17_matrix.csv')
'''


### NOTE ON CONVERSIONS =================================
# Using ISA-LSMS 17 I didnt have crop-units conversions for several units. What I manually did in a excel file was the following:
# 1. Check if missing units are the ones from upper numbers (above 25)
# 2. Use conversion units from the production side for the crop-units possible: pail (large), 50kg bag, 90kg bag, and Ox chart.
# 3. Use conversion units from the consumption side of an older ISA-LSMS (15): bale, basket (shelled), basket (unshelled), cup, tin.
conversionkg_pivot = pd.read_csv('conversionkg_final_matrix.csv',  index_col=0)

#4.  All units have at least one crop conversion. To fill the whole matrix I use the median value
conversionkg_pivot = conversionkg_pivot.apply(lambda x: x.fillna(x.median()),axis=1)

conversion_median =conversionkg_pivot.median(axis=1).to_frame()
conversion_median.columns =['conversionkg']
conversion_median.to_csv('conversions/median_conversions_kg.csv')
# =============================================================================
# Generate empty variables
# =============================================================================

#Obtain the names of the variables per each question of item. Question c is monetary question so not conversion to kgs needed
a_var = []
b_var = []
c_var = []
d_var = []

#Generate variable lables in a list
for item in list_items :
    a = item+'_a'
    b= item+'_b'
    c = item+'_c'  ## expenditure 
    d = item+'_d'
   
    a_var.append(a)
    b_var.append(b)
    c_var.append(c)
    d_var.append(d)
    
list_questions = ['a','b','d']


# check question on whether did something in return and what
# very few households reported having to do something in return

'''
sum_outreturn_yes = []
sum_outreturn_what = []
sum_inreturn_yes = []
sum_inreturn_what = []

for item in list_items :
   a= data[item+'_c_trout_1'].describe()
   print(a)
   sum_outreturn_yes.append(a)
    
for item in list_items :
  a= data[item+'_d_trout_1'].describe()
  print(a)
  sum_outreturn_what.append(a)
  
for item in list_items :
  a= data[item+'_c_trin_1'].describe()
  print(a)
  sum_inreturn_yes.append(a)
   
for item in list_items :
  a= data[item+'_d_trin_1'].describe()
  print(a)
  sum_inreturn_what.append(a)   
'''
  
# convert all empty observations to 0. I do that to convert empty units to 99. If not, main conversion loop doesn't work.
# Note that empty doesn't necessary mean 0, so we careful at looking the data       
data = data.stack().apply(pd.to_numeric, errors='ignore').fillna(0).unstack()


# Drop nan observations. Also drop unit 25 (number not in our choices). Also drop 24(whole chicken). Drop empty cells observations
print('UNITS SELECTED PER ITEM')


#Find the households-questions that reported other units.
df_other_units = pd.DataFrame(columns=['householdid','question', 'other_unit']) 
for var in list_items:
    for i in range(1,4):   #Loop over unit questions.       
        # Find who said other units
        other_units_guy = data.loc[data[var+'_unit'+str(i)]=='other', ['householdid', var+'_unit'+str(i), var+'_unit'+str(i)+'_other']]            
        if other_units_guy.empty:
            continue
        else:
            d = {'householdid': other_units_guy.iloc[:,0], 'question': other_units_guy.columns[1], 'other_unit':other_units_guy.iloc[:,2]}   
            row = pd.DataFrame(data=d)
            df_other_units = df_other_units.append(row)
        
df_other_units['kg'] = np.nan   
df_other_units.to_csv('other units/other_units_consumption.csv')
print('All households-item-question combinations that reported "other" units')         
print(df_other_units)

df_other_units2 = pd.read_excel('other units/other_units_consumption_conversion.xlsx', engine='openpyxl')

### add leandro conversions:

#df_other_units = df_other_units[['householdid',,'other_unit','kg']]

for var in list_items:
    for i in range(1,4):   #Loop over unit questions.
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace('other', 100)
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace(np.nan, 99)
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace(25, 99)
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace(23, 99)
        data[[var+'_unit'+str(i)]] = data[[var+'_unit'+str(i)]].replace(0, 99)
        
        
        

#%% =============================================================================
#   Convert to kgs:  
# =============================================================================

# Generate kg variables empty
for item in list_items:
    for q in list_questions:
        data[item+'_'+q+'kg']= np.nan
        
        
print('a: Total Consumption')
for var in a_var:
    item = var[:-2]
    for i in range(len(data)):
        data.iloc[i,data.columns.get_loc(var+'kg')] = data.iloc[i,data.columns.get_loc(var)]*conversionkg_pivot.loc[int(data.iloc[i,  data.columns.get_loc(item+'_unit1')]), item]
    print(data[[var+'kg']].describe())

print('b: Bought')
for var in b_var:
    item = var[:-2]
    print(var)
    for i in range(len(data)): 
        data.iloc[i,data.columns.get_loc(var+'kg')] = data.iloc[i,data.columns.get_loc(var)]*conversionkg_pivot.loc[int(data.iloc[i,  data.columns.get_loc(item+'_unit2')]), item]

print('d: Own-produced')
for var in d_var:
    item = var[:-2]
    for i in range(len(data)): 
        data.iloc[i,data.columns.get_loc(var+'kg')] = data.iloc[i,data.columns.get_loc(var)]*conversionkg_pivot.loc[int(data.iloc[i,  data.columns.get_loc(item+'_unit3')]), item]




# Replace extreme values for median (let's be careful with this)
# WE MIGHT WANT TO CHANGE THESE CORRECTIONS OF EXTREME VALUES
data.loc[data['sugar_akg']>5,['sugar_akg']] = data['sugar_akg'].median()
data.loc[data['mandazidou_akg']>10,['mandazidou_akg']] = data['mandazidou_akg'].median()
data.loc[data['thobwa_akg']>30,['thobwa_akg']] = data['thobwa_akg'].median()
data.loc[data['tomato_akg']>10,['tomato_akg']] = data['tomato_akg'].median()
data.loc[data['tanaposi_akg']>10,['tanaposi_akg']] = data['tomato_akg'].median()


data.loc[data['maizemgaiwa_akg']>60,['maizemgaiwa_akg']] = data['maizemgaiwa_akg'].median()
data.loc[data['maizerefined_akg']>60,['maizerefined_akg']] = data['maizerefined_akg'].median()
data.loc[data['maizegrain_akg']>30,['maizegrain_akg']] = data['maizegrain_akg'].median()
data.loc[data['wsweetpotatoes_akg']>30,['wsweetpotatoes_akg']] = data['wsweetpotatoes_akg'].median()
data.loc[data['bbean_akg']>10,['bbean_akg']] = data['bbean_akg'].median()
data.loc[data['fleshfish_akg']>10,['fleshfish_akg']] = data['fleshfish_akg'].median()
data.loc[data['banana_akg']>10,['banana_akg']] = data['banana_akg'].median()
data.loc[data['sugarcane_akg']>10,['sugarcane_akg']] = data['sugarcane_akg'].median()
data.loc[data['cookingoil_akg']>10,['cookingoil_akg']] = data['cookingoil_akg'].median()


### ADD THE OTHER UNITS FOUND BY LEANDRO
df_other_units2['question'] = df_other_units2['question'].astype(str)

df = df_other_units2.drop_duplicates(subset=['householdid','question']) # there are no duplicates so it is easy
bb = data[['maizemgaiwa_a', 'maizemgaiwa_unit1', 'maizemgaiwa_akg']]

## for total quantity
for item in list_items:
    for hh in df_other_units2.householdid.unique():
        data.loc[(data[item+'_unit1']==100) & (data['householdid']==hh),item+'_bkg'] = np.array(df_other_units2.loc[(df_other_units2['question']==item+'_unit1') & (df_other_units2['householdid']==hh), 'kg'])*np.array(data.loc[(data[item+'_unit1']==100) & (data['householdid']==hh),item+'_b'])

# for quantity boughy  
for item in list_items:
    for hh in df_other_units2.householdid.unique():
        data.loc[(data[item+'_unit2']==100) & (data['householdid']==hh),item+'_bkg'] = np.array(df_other_units2.loc[(df_other_units2['question']==item+'_unit2') & (df_other_units2['householdid']==hh), 'kg'])*np.array(data.loc[(data[item+'_unit2']==100) & (data['householdid']==hh),item+'_b'])


data.loc[data['sugar_bkg']>5,['sugar_bkg']] = data['sugar_bkg'].median()
data.loc[data['mandazidou_bkg']>10,['mandazidou_bkg']] = data['mandazidou_bkg'].median()
data.loc[data['thobwa_bkg']>30,['thobwa_bkg']] = data['thobwa_bkg'].median()
data.loc[data['tomato_bkg']>10,['tomato_bkg']] = data['tomato_bkg'].median()
data.loc[data['tanaposi_bkg']>10,['tanaposi_bkg']] = data['tomato_bkg'].median()


data.loc[data['maizemgaiwa_bkg']>60,['maizemgaiwa_bkg']] = data['maizemgaiwa_bkg'].median()
data.loc[data['maizerefined_bkg']>60,['maizerefined_bkg']] = data['maizerefined_bkg'].median()
data.loc[data['maizegrain_bkg']>30,['maizegrain_bkg']] = data['maizegrain_bkg'].median()
data.loc[data['wsweetpotatoes_bkg']>30,['wsweetpotatoes_bkg']] = data['wsweetpotatoes_bkg'].median()
data.loc[data['bbean_bkg']>10,['bbean_bkg']] = data['bbean_bkg'].median()
data.loc[data['fleshfish_bkg']>10,['fleshfish_bkg']] = data['fleshfish_bkg'].median()
data.loc[data['banana_bkg']>10,['banana_bkg']] = data['banana_bkg'].median()
data.loc[data['sugarcane_bkg']>10,['sugarcane_bkg']] = data['sugarcane_bkg'].median()
data.loc[data['cookingoil_bkg']>10,['cookingoil_bkg']] = data['cookingoil_bkg'].median()






# for quantity own-produced 
for item in list_items:
    for hh in df_other_units2.householdid.unique():
        data.loc[(data[item+'_unit3']==100) & (data['householdid']==hh),item+'_bkg'] = np.array(df_other_units2.loc[(df_other_units2['question']==item+'_unit3') & (df_other_units2['householdid']==hh), 'kg'])*np.array(data.loc[(data[item+'_unit3']==100) & (data['householdid']==hh),item+'_b'])
 

print('===============================================================')         
print('a: Total Consumption (in kg) after corrections (cleaned)')
print('===============================================================') 

for var in a_var:
    item = var[:-2]
    print(data[[var+'kg']].describe())






#%%

#Check 
print('To check if the conversion to kgs works correctly: check for example for beans')           
print(data[['bbean_a', 'bbean_unit1', 'bbean_akg','bbean_b', 'bbean_unit2',  'bbean_bkg', 'bbean_d', 'bbean_unit3', 'bbean_dkg']])


#check total consumption in kgs
data['total_foodkg'] = 0
data['purchased_kg'] = 0
data['ownproduced_kg'] = 0

for item in list_items:
    data['total_foodkg'] += data[item+'_akg'].replace(np.nan, 0)
    data['purchased_kg'] += data[item+'_bkg'].replace(np.nan, 0)
    data['ownproduced_kg'] += data[item+'_dkg'].replace(np.nan, 0)
    
    
sumtotalfoodkg = data[['total_foodkg', 'purchased_kg','ownproduced_kg']].describe()   

print('==== Summary Food Consumption last 7 days in kgs aggregated across items =======')
print('')
print(sumtotalfoodkg)


#%%  CONVERT TO MONETARY VALUE

### CHECK PRICES FROM ISA-LSMS

p_isalsms = pd.read_stata('prices/price_consumption_kg.dta')
p_isalsms = p_isalsms.loc[(p_isalsms['region']=='Southern')& (p_isalsms['year']==2017)]
p_isalsms = p_isalsms.groupby(by=['crop_code']).median()

p_isalsms_sell = pd.read_stata('prices/price_prioduction_kg.dta')
p_isalsms_sell = p_isalsms_sell.groupby(by=['crop_code']).median()


# Generate price variables
for item in list_items:
        data[item+'_price']= np.nan


list_items.remove('mango')
list_items.remove('fingermillet')
# price per household
for item in list_items:
    data[item+'_price'] = data[item+'_c'] / data[item+'_bkg'].replace(0,np.nan)
    
    print('Check: Distribution of price '+item)
    
    print(data[[item+'_price']].describe())
    


price_data = pd.DataFrame(list_items, columns=['good'])
price_data['p_c'] = np.nan

for item in list_items:
    print('Median Price 1 kg of '+item)
    data['med_price_'+item] = data[item+'_price'].median()
    print(data['med_price_'+item].mean())    
    price_data.loc[price_data['good']==item,'p_c'] = data['med_price_'+item].mean()

data['med_price_mango'] = data['med_price_guava']   ### I couldnt find a price for mango in isa-lsms. I use guava price which is a similar fruit.
data['med_price_fingermillet'] = 1653   ### I use the price from the ISA-LSMS 17 (290) and multiply it for the change of maize price ISA-LSMS vs village19


price_data.to_csv('prices/village_c_prices.csv', index=False)
## compute consumption in MWK
    
for item in list_items:
    for q in list_questions:
        data[item+'_'+q+'MWK']= np.nan

print('a: Total Consumption')
for item in list_items:
    print(item)
    data[item+'_aMWK'] = data[item+'_akg']*data['med_price_'+item]
    print('Food Consumption in MWK during last 7 days item: '+item)
    print(data[item+'_aMWK'].describe(percentiles=percentiles))
            
print('b: Bought')
for item in list_items:
    print(item)
    data[item+'_bMWK'] = data[item+'_bkg']*data['med_price_'+item]

### I noticed some outliers in quantites consumed: remove them
for item in ['tomato', 'fleshfish', 'sugar', 'cookingoil', 'wsweetpotatoes']:
    data[[item+'_akg', item+'_aMWK']] = remove_outliers(data[[item+'_akg', item+'_aMWK']],hq=0.99)

data[['sugar_akg', 'sugar_aMWK']] = remove_outliers(data[['sugar_akg', 'sugar_aMWK']],hq=0.95)

for item in ['tomato', 'fleshfish', 'sugar', 'cookingoil', 'wsweetpotatoes']:
    data[[item+'_bkg', item+'_bMWK']] = remove_outliers(data[[item+'_bkg', item+'_bMWK']],hq=0.99)
    
for item in ['tomato', 'fleshfish', 'sugar', 'cookingoil', 'wsweetpotatoes']:
    data[[item+'_dkg', item+'_dMWK']] = remove_outliers(data[[item+'_dkg', item+'_dMWK']],hq=0.99)
    
    
print('d: Own-produced')
for item in list_items:
    #print(item)
    data[item+'_dMWK'] = data[item+'_dkg']*data['med_price_'+item]
    
    
#check total consumption
data['c_food'] = 0
data['c_food_purch'] = 0
data['c_food_ownprod'] = 0

for item in list_items:
    data['c_food'] += data[item+'_aMWK'].replace(np.nan, 0)
    data['c_food_purch'] += data[item+'_bMWK'].replace(np.nan, 0)
    data['c_food_ownprod'] += data[item+'_dMWK'].replace(np.nan, 0)

data[['c_food', 'c_food_purch' , 'c_food_ownprod']] = data[['c_food', 'c_food_purch' , 'c_food_ownprod']].replace(0.0,np.nan)



pd.options.display.float_format = '{:,.2f}'.format
sumcfood= ((data[['c_food', 'c_food_purch' , 'c_food_ownprod']]*4*7/dollar_MWK).replace(0.0,np.nan)).describe(percentiles)  
print('==== Summary Food Consumption at rainy in dollars =======')
print(sumcfood) 


#%% non-food consumption  (month)


### check if 3 months window consumption is lower

data['c_housing'] = data['nonf_cons_a_2']
data['c_clothes'] = data['nonf_cons_b_2'] 
data['c_education'] = data['nonf_cons_c_2'] 
data['c_health'] = data['nonf_cons_d_2']
data['c_funeralout'] = data['nonf_cons_e_2']
data['c_funeralin'] = data['nonf_cons_f_2']
data['c_weddingout'] = data['nonf_cons_g_2']
data['c_weddingin'] = data['nonf_cons_h_2'] 

data[['c_food_purch', 'c_housing']] = remove_outliers(data[['c_food_purch', 'c_housing']], hq=0.99)


data['c_nonfood'] = data[['c_housing', 'c_clothes', 'c_education', 'c_health', 'c_funeralout', 'c_funeralin', 'c_weddingout', 'c_weddingin']].sum(axis = 1, skipna = True)

sum_cnonfood = ((data[['c_nonfood','c_housing', 'c_clothes', 'c_education', 'c_health', 'c_funeralout', 'c_funeralin', 'c_weddingout', 'c_weddingin']]/(3*dollar_MWK)).replace(0.0,np.nan)).describe()
print('======== SUMMARY  NON-FOOD CONSUMPTION (MONTH LEVEL)')
print('summary in MWK')
sum_cnonfood


#### for non-food consumption we have the questions duplicated at last month and last 3 months. Last 3 months gets a lower yearly value. Evidence consumption after harvest is larger



#%% SAVE DATASET 

# Export dataset
data['hhid'] = data['householdid']


for item in list_items:
    data['c_'+item+'_kg_7days'] = data[item+'_akg']
    data['c_'+item+'_MWK_7days'] = data[item+'_aMWK']



#data[['c_food_purch','c_food_ownprod']] = remove_outliers(data[['c_food_purch','c_food_ownprod']],hq=0.99)

## short dataset
datacon_short = data[['hhid','c_food','total_foodkg','c_food_purch','c_food_ownprod', 'c_nonfood','c_housing', 'c_clothes', 'c_education', 'c_health', 'c_funeralout', 'c_funeralin', 'c_weddingout', 'c_weddingin']]

## Food at 3 months level
datacon_short[['c_food','total_foodkg','c_food_purch','c_food_ownprod']] = datacon_short[['c_food','total_foodkg','c_food_purch','c_food_ownprod']]*4*3

datacon_short['ctotal'] = datacon_short[['c_nonfood', 'c_food']].sum(axis = 1, skipna = True)


## Consumption at year level
datacon_short[['ctotal','c_food','total_foodkg','c_food_purch','c_food_ownprod', 'c_nonfood','c_housing', 'c_clothes', 'c_education', 'c_health']] = datacon_short[['ctotal','c_food','total_foodkg','c_food_purch','c_food_ownprod', 'c_nonfood','c_housing', 'c_clothes', 'c_education', 'c_health']]*(12/3)


c_summary =  ((datacon_short[['ctotal','c_food','c_food_purch','c_food_ownprod', 'c_nonfood','c_housing', 'c_clothes', 'c_education', 'c_health']]/dollar_MWK).replace(0.0,np.nan)).describe(percentiles=percentiles)

print(c_summary)

datacon_short.to_csv('cons_short_19.csv', index=False)


##Long dataset (at rainy season)
data.to_csv('cons_long_19.csv', index=False)


UNITS SELECTED PER ITEM
All households-item-question combinations that reported "other" units
    householdid              question other_unit  kg
58           68     maizemgaiwa_unit1        5kg nan
58           68     maizemgaiwa_unit2        5kg nan
163         189      maizegrain_unit1    Handful nan
203         236      maizegrain_unit1    Handful nan
197         230  wsweetpotatoes_unit1       Pots nan
..          ...                   ...        ...  ..
74           85   locallybrewed_unit2       25ml nan
16           21            salt_unit1     Packet nan
166         192            salt_unit1    Handful nan
16           21            salt_unit2     Packet nan
166         192            salt_unit2    Handful nan

[81 rows x 4 columns]
a: Total Consumption
       maizemgaiwa_akg
count           183.00
mean             11.02
std               9.04
min               0.71
25%               5.00
50%               9.60
75%              15.56
max              67.20
       maizerefined

maizemadeya_b
maizegrain_b
greenmaize_b
rice_b
cassavatubers_b
wsweetpotatoes_b
osweetpotatoes_b
ipotatoes_b
potatocrisps_b
bbean_b
pigeonpea_b
groundnut_b
groundnutf_b
onion_b
cabbage_b
tanaposi_b
leafyvegetables_b
tomato_b
eggs_b
driedfish_b
fleshfish_b
goat_b
chicken_b
otherpoultry_b
smockedfish_b
mango_b
banana_b
guava_b
wildfruits_b
sugar_b
sugarcane_b
cookingoil_b
softdrinks_b
thobwa_b
locallybrewed_b
salt_b
fingermillet_b
mandazidou_b
d: Own-produced
a: Total Consumption (in kg) after corrections (cleaned)
       maizemgaiwa_akg
count           183.00
mean             10.71
std               8.02
min               0.71
25%               5.00
50%               9.60
75%              15.56
max              46.67
       maizerefined_akg
count            164.00
mean               9.42
std                7.14
min                0.87
25%                4.50
50%               10.00
75%               12.50
max               37.50
       maizemadeya_akg
count           109.00
mean        

Check: Distribution of price maizemgaiwa
        maizemgaiwa_price
count               72.00
unique              38.00
top                240.00
freq                14.00
Check: Distribution of price maizerefined
        maizerefined_price
count                40.00
unique               25.00
top                 240.00
freq                  7.00
Check: Distribution of price maizemadeya
        maizemadeya_price
count               21.00
unique              17.00
top                200.00
freq                 3.00
Check: Distribution of price maizegrain
        maizegrain_price
count              18.00
unique             15.00
top               400.00
freq                2.00
Check: Distribution of price greenmaize
        greenmaize_price
count              19.00
unique              9.00
top               296.91
freq                5.00
Check: Distribution of price rice
        rice_price
count       101.00
unique       39.00
top         560.00
freq         16.00
Check: Distribution of

count     199.00
mean      172.91
std       149.50
min        21.01
5%         70.03
10%        70.03
25%        70.03
50%       105.31
75%       200.00
80%       200.00
90%       210.19
95%       402.02
99%       702.27
max     1,000.00
Name: groundnutf_aMWK, dtype: float64
onion
Food Consumption in MWK during last 7 days item: onion
count   149.00
mean    162.58
std     120.69
min      50.00
5%       50.00
10%      50.00
25%     100.00
50%     150.00
75%     200.00
80%     200.00
90%     300.00
95%     350.00
99%     569.33
max     950.00
Name: onion_aMWK, dtype: float64
cabbage
Food Consumption in MWK during last 7 days item: cabbage
count   111.00
mean    271.00
std     161.59
min      40.06
5%       69.77
10%     139.55
25%     200.00
50%     200.00
75%     400.00
80%     400.00
90%     600.00
95%     600.00
99%     780.00
max     800.00
Name: cabbage_aMWK, dtype: float64
tanaposi
Food Consumption in MWK during last 7 days item: tanaposi
count     216.00
mean      195.45
std      

d: Own-produced
==== Summary Food Consumption at rainy in dollars =======
        c_food  c_food_purch  c_food_ownprod
count   252.00        251.00          243.00
mean    347.63        800.14          166.44
std     166.03      9,635.06          218.56
min      62.26          5.91            1.91
5%      131.74         38.09            7.64
10%     156.97         49.58           15.03
25%     217.13         86.81           56.65
50%     328.23        166.74          130.91
75%     440.17        249.39          227.27
80%     465.68        267.77          251.44
90%     562.12        342.99          307.75
95%     671.76        449.29          391.40
99%     822.58        610.89          665.89
max   1,013.35    152,813.01        2,873.05
======== SUMMARY  NON-FOOD CONSUMPTION (MONTH LEVEL)
summary in MWK
        ctotal   c_food  c_food_purch  c_food_ownprod  c_nonfood  c_housing  \
count   252.00   252.00        245.00          243.00     250.00     243.00   
mean    728.85   595.94  